In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.0 Image generation in Vertex AI with REST API

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_0_image_gen_rest_api.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgetting-started%2Fintro_gemini_2_0_image_gen_rest_api.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/getting-started/intro_gemini_2_0_image_gen_rest_api.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_0_image_gen_rest_api.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_0_image_gen_rest_api.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_0_image_gen_rest_api.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_0_image_gen_rest_api.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_0_image_gen_rest_api.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_0_image_gen_rest_api.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Author(s) |
| --- |
| [Nikita Namjoshi](https://github.com/nikitamaia) |

## Overview

In this tutorial, you learn how to use Gemini 2.0's image generation features in Vertex AI using the REST API.

You'll try out the following scenarios:
* text --> image
* text --> image + text (interleaved)
* text + image --> image


## Getting Started

### Install required libraries

In [1]:
%%capture

!sudo apt install -q jq

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [3]:
# Use the environment variable if the user doesn't provide Project ID.
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

## Use the Gemini 2.0 Flash model

In [4]:
MODEL_ID = "gemini-2.0-flash-exp"  # @param {type: "string"}

### Defining environment variables for cURL commands

These environment variables are used to construct the cURL commands.

In [5]:
import os

os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["LOCATION"] = LOCATION

API_HOST = f"{LOCATION}-aiplatform.googleapis.com"
os.environ["API_ENDPOINT"] = (
    f"{API_HOST}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/publishers/google/models/{MODEL_ID}"
)

## Generate Text + Image

First, send a text prompt to Gemini 2.0 describing the image you want generated.


In the curl command below, you'll see that the payload includes the following keys:

* `contents`: this is your prompt, in this case a text only user message
*`generation_config`: this dictionary specifies the desired output modalities, in this case `TEXT` and `IMAGE`. If you do not specify `IMAGE`, you will not get image output.

The cell below writes the output of running the curl command to the file `response.json`.

In [38]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": { "text": "generate an image of a grackle wearing a top hat and monocle."},
    },
    "generation_config": {
      "response_modalities": ["TEXT", "IMAGE"],
     },
  }' 2>/dev/null >response.json

Let's examine the output in the `response.json` file.

In `content` you can see the model has created an `image/png` which is the b64 encoded value to the `data` key.

In [39]:
!cat response.json

Next, load in the data from the `response.json` file so it's easier to work with in Python.

In [40]:
import json

with open("response.json") as f:
    response_data = json.load(f)
    print(response_data)

Extract the image data from the response and visualize.

In [41]:
import base64

from IPython.display import Image, display

image_part = next(
    filter(
        lambda x: "inlineData" in x,
        response_data["candidates"][0]["content"]["parts"],
    )
)

image_data = base64.b64decode(image_part["inlineData"]["data"])
display(Image(data=image_data))

## Generate Text + Image

In addition to generating images, Gemini can generate multiple images and text in an interleaved fashion.

For example, you could ask the model to generate a recipe for banana bread with image showing different stages of the cooking process. Our you could ask the model to generate a images of different wildflowers with accompanying titles and descriptions.

Let's try out the interleaved text+image functionality by prompting Gemini 2.0 to create an illustrated children's story.

Here's the prompt:

*Make a children's storybook about a curious young fox named Mosi, who sets off on a magical adventure through a forest in search of a special star. The story unfolds over three episodes, with each episode introducing Mosi to a new friend and revealing wondrous and magical landscapes.  For each episode, provide a title, a captivating narrative, and also generate a realistic image illustrating everything in the scene described in the narrative of that episode.*

You'll notice that in the prompt we ask the model to generate both text and images for each episode of the narrative. This will nudge the model to create text with images interleaved.

⚠️ Note that we are asking the model to generate a lot of content in this prompt, so it will take a bit of time for this cell to finish executing.

In [22]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": { "text": "Make a children's storybook about a curious young fox named Mosi, who sets off on a magical adventure through a forest in search of a special star.  The story unfolds over three episodes, with each episode introducing Mosi to a new friend and revealing wondrous and magical landscapes.  For each episode, provide a title, a captivating narrative, and also generate a realistic image illustrating everything in the scene described in the narrative of that episode."},
    },
    "generation_config": {
      "response_modalities": ["TEXT", "IMAGE"],
     },
  }' 2>/dev/null >response.json

This time, when we look at the model response, you'll see 3 `text` values with 3 b64 images (`inlineData`) interleaved.

In [23]:
!jq -C ".candidates[].content" response.json

Let's visualize the response.

In [24]:
with open("response.json") as f:
    response_data = json.load(f)

In [25]:
for part in response_data["candidates"][0]["content"]["parts"]:
    if "text" in part.keys():
        print(part["text"])
    if "inlineData" in part.keys():
        content = part["inlineData"]["data"]
        image_data = base64.b64decode(content)
        display(Image(data=image_data))

## Generate a new image from a text + image prompt
You can pass text and an image to Gemini 2.0 for use cases like product captions, information about a particular image, or to make edits or modifications to an image.

Let's try out a style transfer example and ask Gemini 2.0 to create an image of this dog in the baroque style.

![small-dog-pink](https://storage.googleapis.com/cloud-samples-data/generative-ai/image/small-dog-pink.jpg)

In [30]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": [
        {"file_data": {
          "mime_type": "image/png",
          "file_uri": "gs://cloud-samples-data/generative-ai/image/small-dog-pink.jpg"
          }
        },
        {"text": "Generate a baroque style portrait painting of this dog."},
      ]

    },
    "generation_config": {
      "response_modalities": ["TEXT", "IMAGE"],
     },
  }' 2>/dev/null >response.json

Extract the image data from the response and visualize.

In [37]:
with open("response.json") as f:
    response_data = json.load(f)

image_part = next(
    filter(
        lambda x: "inlineData" in x,
        response_data["candidates"][0]["content"]["parts"],
    )
)

image_data = base64.b64decode(image_part["inlineData"]["data"])
display(Image(data=image_data))